In [1]:
import pandas as pd

#______________________________________________________________________
#Ќеобходимые операции с матрицами и векторами

#ѕеремножение матрицы и вектора
def multplMatrVect(A, x):
    c = [0 for rows in range(len(A))]
    for i in range(len(x)):
        for j in range(len(x)):
            c[i] = c[i] + A[i][j]*x[j]
    return c
    


#”множение векторов
def multplVect(a, b):
    res = 0
    for i in range(len(a)):
            res += a[i]*b[i]
    return res


#”множение вектора на скал€р
def IntMultVect(alpha, x):
  c = [0 for i in range(len(x))]
  for i in range(len(x)):
    c[i]=alpha*x[i]
  return c


#—умма векторов
def sumVect(a,b):
  c = [0 for i in range(len(a))]
  for i in range(len(a)):
    c[i]=a[i]+b[i]
  return c



#»терационна€ формула √ерона
def GeronRoot(c, eps=10**(-6)/3):
  x0=1 if c<1 else c
  x1= 1/2*(x0+c/x0)
  while abs(x1-x0)>eps:
    x0=x1
    x1= 1/2*(x1+c/x1)
  return x1


#≈вклидова норма вектора
def vectorNorm(x):
  total = 0
  for i in range(len(x)):
    total+=(x[i])**2
  return GeronRoot(total)


#______________________________________________________________________
# ћетод наискорейшего градиентного спуска

#ѕриводим данную функцию к квадратичной, учитыва€ N = 13 и 
#находим минимальное собственное число матрицы ј
A = [[4, 1, 1], [1, 8.6, -1], [1, -1, 10.6]]
b = [1, -2, 3]
eig_min = 3.589
eps = 10**(-6)


#¬ыбираем начальное приближение и вычисл€ем необходимые дл€
#следующего шага компоненты из формулы (2.2)
x_prev = [2, 3, 4]
q = sumVect(multplMatrVect(A, x_prev), b)  #A*x_k+b
m = -(vectorNorm(q))**2/(multplVect(q, multplMatrVect(A, q)))
x_next = sumVect(x_prev, IntMultVect(m, q))

#—читаем значение функции в начальной точке и получившейс€
f_prev = 0.5*multplVect(x_prev, multplMatrVect(A, x_prev))+multplVect(x_prev, b)+13
f_next = 0.5*multplVect(x_next, multplMatrVect(A, x_next))+multplVect(x_next, b)+13


#ƒобавл€ем эти значени€ в нашу таблицу
table = pd.DataFrame(columns=('x', 'y', 'z', 'f(x, y, z)'))
pd.options.display.float_format = "{:,.8f}".format

new_row = {'x': x_prev[0], 'y': x_prev[1], 'z': x_prev[2], 'f(x, y, z)':f_prev}
table = table.append(new_row, ignore_index = True)
new_row = {'x': x_next[0], 'y': x_next[1], 'z': x_next[2], 'f(x, y, z)':f_next}
table = table.append(new_row, ignore_index = True)


#ѕроводим аналогичные действи€ до тех пор, пока права€ часть
#формулы (2.6) не будет удовлетвор€ть заданной точности
while vectorNorm(q)*eig_min > eps:
    x_prev = x_next
    
    q = sumVect(multplMatrVect(A, x_prev), b)
    m = -(vectorNorm(q))**2/(multplVect(q, multplMatrVect(A, q)))
    
    x_next = sumVect(x_prev, IntMultVect(m, q))
    f_next = 0.5*multplVect(x_next, multplMatrVect(A, x_next))+multplVect(x_next, b)+13
    
    new_row = {'x': x_next[0], 'y': x_next[1], 'z': x_next[2], 'f(x, y, z)':f_next}
    table = table.append(new_row, ignore_index = True)

table


#______________________________________________________________________
# ћетод наискорейшего покоординатного спуска

#ѕриводим данную функцию к квадратичной, учитыва€ N = 13 и 
#находим минимальное собственное число матрицы ј
A = [[4, 1, 1], [1, 8.6, -1], [1, -1, 10.6]]
b = [1, -2, 3]
eig_min = 3.589
eps = 10**(-6)


#¬ыбираем начальное приближение и вычисл€ем необходимые дл€
#следующего шага компоненты 
x_prev = [2, 3, 4]
q = sumVect(multplMatrVect(A, x_prev), b) #A*x_k+b
f_prev = 0.5*multplVect(x_prev, multplMatrVect(A, x_prev))+multplVect(x_prev, b)+13

#‘ормируем таблицу и добавл€ем туда начальное значение    
table = pd.DataFrame(columns=('x', 'y', 'z', 'f(x, y, z)'))
pd.options.display.float_format = "{:,.8f}".format
new_row = {'x': x_prev[0], 'y': x_prev[1], 'z': x_prev[2], 'f(x, y, z)':f_prev}
table = table.append(new_row, ignore_index = True)
    

#”словие остановки такое же, как в предыдущем методе
while vectorNorm(q)*eig_min > eps:
    #¬водим вектор e_i дл€ попеременных шагов по координатам x, y, z
    for i in range(0, 3):
        q = sumVect(multplMatrVect(A, x_prev), b)
        e_i = [0, 0, 0]
        e_i[i] = 1  
        m = - (multplVect(e_i, q))/(multplVect(e_i, multplMatrVect(A, e_i)))

        #делаем очередной шаг и просчитываем значение функции в этой точке
        x_next = sumVect(x_prev, IntMultVect(m, e_i))
        f_next = 0.5*multplVect(x_next, multplMatrVect(A, x_next))+multplVect(x_next, b)+13

        #добавл€ем данные в таблицу
        new_row = {'x': x_next[0], 'y': x_next[1], 'z': x_next[2], 'f(x, y, z)':f_next}
        table = table.append(new_row, ignore_index = True)
        
        x_prev = x_next
    
table



,x,y,z,"f(x, y, z)"
0,2.00000000,3.00000000,4.00000000,154.50000000
1,-2.00000000,3.00000000,4.00000000,122.50000000
2,-2.00000000,0.93023256,4.00000000,104.07906977
3,-2.00000000,0.93023256,-0.00658183,18.99977040
4,-0.48091268,0.93023256,-0.00658183,14.38451784
5,-0.48091268,0.28771289,-0.00658183,12.60934228
6,-0.48091268,0.28771289,-0.21050702,12.38893922
7,-0.26930147,0.28771289,-0.21050702,12.29938061
8,-0.26930147,0.23939470,-0.21050702,12.28934163
9,-0.26930147,0.23939470,-0.23502866,12.28615468
